In [1]:
import MetaTrader5 as mt5
import pandas as pd
import datetime
from functions import isred,isgreen
import numpy as np
import itertools



In [2]:
mt5.initialize()

True

In [21]:
def get_data(currency):
    
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2021, 2, 1), datetime.datetime.now()))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data

In [4]:

def rsi( data, periods):
    for period in periods:
        # Calculate the change in price
        delta = data['Close'].diff()

        # Make gains and losses series
        up = delta.clip(lower=0)
        down = -1 * delta.clip(upper=0)

        # Calculate the average gain and average loss
        avg_gain = up.rolling(window=period).mean()
        avg_loss = down.rolling(window=period).mean()

        # Calculate the relative strength
        rs = avg_gain / avg_loss

        # Calculate the RSI
        rsi = 100 - (100 / (1 + rs))

        # Add the RSI values as a new column with period number
        data['rsi_{}'.format(period)] = rsi
    return data


In [5]:
def calculate_sma(data, periods):
    for period in periods:
        sma_values = []
        close_prices = data['Close'].values

        for i in range(period, len(data) + 1):
            sma = sum(close_prices[i - period:i]) / period
            sma_values.append(sma)

        data[f'sma_{period}'] = [None] * (period - 1) + sma_values

    return data

In [22]:
data=get_data("GOLD")

In [23]:
data

,time,Open,High,Low,Close,tick_volume,spread,real_volume
0,1612141200,1862.93,1866.92,1851.62,1853.43,1232,36,0
1,1612142100,1853.53,1859.75,1851.94,1857.41,810,36,0
2,1612143000,1857.38,1859.85,1855.34,1856.60,852,36,0
3,1612143900,1856.83,1857.75,1854.97,1855.83,810,36,0
4,1612144800,1855.61,1856.31,1852.00,1853.02,881,36,0
...,...,...,...,...,...,...,...,...
65780,1699556400,1960.46,1964.01,1959.99,1963.47,1614,26,0
65781,1699557300,1963.46,1963.57,1960.35,1961.49,1467,26,0
65782,1699558200,1961.49,1963.02,1960.23,1960.46,1398,28,0
65783,1699559100,1960.46,1961.96,1960.26,1961.76,1387,29,0


In [7]:
def bolinger(data,std,tpw,s_num):
    k = std  # Number of standard deviations
    df=data
# Calculate typical price
    df['typical_price'] = (df['High'] + df['Low'] + df['Close']) / 3

# Calculate standard deviation
    df['std_dev'] = df['typical_price'].rolling(window=tpw).std()  # Assuming a window of 5 periods

# Calculate Bollinger Bands using the existing 'sma' column
    df['upper_band'] = df[f'sma_{s_num}'] + k * df['std_dev']
    df['lower_band'] = df[f'sma_{s_num}'] - k * df['std_dev']
    return df

In [ ]:

rsil=[6]
data=rsi(data,rsil)
data


In [ ]:
data=calculate_sma(data,[6])

In [ ]:
std=2
tpw=2
data=bolinger(data,std=std,tpw=tpw,s_num=6)
print(data)

In [ ]:
conditions=[(data.rsi_6 < 30) & (data.Close < data.lower_band),
(data.rsi_6 > 70) & (data.Close > data.upper_band)]
choices = ['Buy', 'Sell']
data ['signal'] = np.select(conditions, choices)
data

In [8]:
def splitdf(data,size):
    splitdata=np.array_split(data, size)
    return splitdata

In [18]:
data

""


In [24]:
data1=splitdf(data,3)
ud=data1[0]
ud


,time,Open,High,Low,Close,tick_volume,spread,real_volume
0,1612141200,1862.93,1866.92,1851.62,1853.43,1232,36,0
1,1612142100,1853.53,1859.75,1851.94,1857.41,810,36,0
2,1612143000,1857.38,1859.85,1855.34,1856.60,852,36,0
3,1612143900,1856.83,1857.75,1854.97,1855.83,810,36,0
4,1612144800,1855.61,1856.31,1852.00,1853.02,881,36,0
...,...,...,...,...,...,...,...,...
21924,1641302100,1804.10,1804.41,1802.27,1803.10,863,35,0
21925,1641303000,1803.12,1803.80,1802.34,1803.28,857,35,0
21926,1641303900,1803.26,1804.24,1802.86,1803.89,817,35,0
21927,1641304800,1803.88,1804.88,1801.53,1801.90,1069,35,0


In [ ]:
data

In [ ]:
data.signal=data.signal.shift()

In [10]:
def calc_prof(enter,close):
    prof=enter-close
    return prof

In [ ]:
# trades = {'dateopened': [], 'dateclosed': [], 'profit': []}

# bposition = False
# sposition = False
# buyprices = 0
# startdate = 0
# enddate = 0
# sellprice = 0
# sell_enddate = 0
# sell_startdate = 0

# for index, row in data.iterrows():
#     if not bposition and row['signal'] == 'Buy':
#         if sposition:
#             sellprice = sellprice
#             exitprice = row.Close
#             sell_enddate = row.time
#             profit = calc_prof(sellprice,exitprice)
#             sposition = False
#             trades['dateclosed'].append(sell_enddate)
#             trades['profit'].append(profit)

#         buyprices = row.Open
#         startdate = row.time
#         bposition = True
#         trades['dateopened'].append(startdate)

#     if bposition and row['signal'] == 'Sell':
#         sellprice = row.Open
#         sell_startdate = row.time
#         exitprice = row.Close
#         enddate = row.time
#         profit = calc_prof(sellprice,buyprices)
#         bposition = False
#         trades['dateclosed'].append(enddate)
#         trades['profit'].append(profit)




In [ ]:
trades = []

bposition = False
sposition = False
buyprices = 0
startdate = 0
enddate = 0
sellprice = 0
sell_enddate = 0
sell_startdate = 0
bbprice = []

for index, row in data.iterrows():
    if not bposition and row['signal'] == 'Buy':
        if sposition==True:
            exitprice = row.Close
            sell_enddate = row.time
            profit = calc_prof(sellprice, exitprice)
            sposition = False
            trades.append(('Sell',startdate, sell_enddate, profit))
        bbprice.append(row.Open)
        buyprices = row.Open
        startdate = row.time
        bposition = True

    if bposition:
        if row['signal'] == 'Sell' or row.shifted < 0.95 * bbprice[-1]:
            sellprice = row.Open
            sell_startdate = row.time
            exitprice = row.Close
            enddate = row.time
            sposition = True
            profit = calc_prof(sellprice, buyprices)
            bposition = False
            trades.append(('Buy',startdate, enddate, profit))




In [ ]:
data

In [ ]:
trades

In [ ]:
df = pd.DataFrame(trades, columns=['Tradetype', 'datedpened', 'dateclosed', 'profit'])

In [ ]:
print(df.profit.max())

In [ ]:
print(df.profit.sum())

In [ ]:
df

In [ ]:
df['dateopened'] = df['dateopened'].apply(lambda x: datetime.datetime.fromtimestamp(x))
df['dateclosed'] = df['dateclosed'].apply(lambda x: datetime.datetime.fromtimestamp(x))

# Calculate the duration of each trade
df['duration'] = df['dateclosed'] - df['dateopened']

# Find the maximum duration


In [ ]:
max_duration = df['duration'].max()
max_row = df[df['duration'] == max_duration]

max_profit = max_row['profit'].values[0]

print("Maximum time taken by the longest trade:", max_duration)
print("profit when the maximum time was taken:", max_profit)

In [ ]:
data['shifted']=data.Close.shift()

In [ ]:
df['dateclosed'] = pd.to_datetime(df['dateclosed'])

# Calculate weekly performance
df['Week'] = df['dateclosed'].dt.to_period("W")
weekly_performance = df.groupby('Week').agg({'profit': ['sum', 'min', 'max']})

# Calculate monthly performance
df['Month'] = df['dateclosed'].dt.to_period("M")
monthly_performance = df.groupby('Month').agg({'profit': ['sum', 'min', 'max']})

print("Weekly Performance:")
print(weekly_performance)

print("\nMonthly Performance:")
print(monthly_performance)

In [ ]:
# Calculate weekly performance
df['Week'] = df['dateclosed'].dt.to_period("W")
weekly_performance = df.groupby('Week').agg({'profit': ['sum', 'min', 'max']})

# Calculate monthly performance
df['Month'] = df['dateclosed'].dt.to_period("M")
monthly_performance = df.groupby('Month').agg({'profit': ['sum', 'min', 'max']})

# Finding best week and best month
best_week = weekly_performance['profit']['sum'].idxmax()
best_week_value = weekly_performance.loc[best_week, ('profit', 'sum')]
best_month = monthly_performance['profit']['sum'].idxmax()
best_month_value = monthly_performance.loc[best_month, ('profit', 'sum')]

# Displaying the results
print("Best Week: {} with total profit {}".format(best_week, best_week_value))
print("Best Month: {} with total profit {}".format(best_month, best_month_value))

# Displaying all weeks and months performance
print("\nWeekly Performance:")
print(weekly_performance)

print("\nMonthly Performance:")
print(monthly_performance)


In [11]:
def evaluate(ls,name):
    df=pd.DataFrame(ls, columns=['Tradetype', 'dateopened', 'dateclosed', 'profit'])
    maxprof=df.profit.max()
    pips=maxprof*100
    totalprof=df.profit.sum()
    print(f'The total profit {name} was :{totalprof}')
    df['dateopened'] = df['dateopened'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    df['dateclosed'] = df['dateclosed'].apply(lambda x: datetime.datetime.fromtimestamp(x))

# Calculate the duration of each trade
    df['duration'] = df['dateclosed'] - df['dateopened']
    max_duration = df['duration'].max()
    max_row = df[df['duration'] == max_duration]
    longtype=max_row['Tradetype'].values[0]
    num_buy_trades = df[df['TradeType'] == 'Buy'].shape[0]

# Finding the total profit for 'Buy' trades
    total_sell_profit = df[df['TradeType'] == 'Sell']['Profit'].sum()
    num_sell_trades = df[df['TradeType'] == 'Sell'].shape[0]

# Finding the total profit for 'Buy' trades
    total_buy_profit = df[df['TradeType'] == 'Buy']['Profit'].sum()

    max_profit = max_row['profit'].values[0]
    print(f'the trade that took most time took {max_duration} and made {max_profit} type:{longtype}')
    print(f'Buy :{num_buy_trades} profit:{total_buy_profit}')
    print(f'Sell:{total_buy_profit} profit:{total_sell_profit}')
    print(f'the max duration was {max_duration}')
    print(f'The trade with the highest profit made {maxprof} pips:{pips} ')

In [ ]:
evaluate(trades,'Bolinger_6')

In [25]:
df=ud
print(df)

             time     Open     High      Low    Close  tick_volume  spread  \
0      1612141200  1862.93  1866.92  1851.62  1853.43         1232      36   
1      1612142100  1853.53  1859.75  1851.94  1857.41          810      36   
2      1612143000  1857.38  1859.85  1855.34  1856.60          852      36   
3      1612143900  1856.83  1857.75  1854.97  1855.83          810      36   
4      1612144800  1855.61  1856.31  1852.00  1853.02          881      36   
...           ...      ...      ...      ...      ...          ...     ...   
21924  1641302100  1804.10  1804.41  1802.27  1803.10          863      35   
21925  1641303000  1803.12  1803.80  1802.34  1803.28          857      35   
21926  1641303900  1803.26  1804.24  1802.86  1803.89          817      35   
21927  1641304800  1803.88  1804.88  1801.53  1801.90         1069      35   
21928  1641305700  1801.85  1802.30  1798.42  1799.44         1390      35   

       real_volume  
0                0  
1                0  


In [ ]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.Low[l]>df1.Low[i]):
            pividlow=0
        if(df1.High[l]<df1.High[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,15,15), axis=1)
df['shortpivot'] = df.apply(lambda x: pivotid(df, x.name,5,5), axis=1)

In [ ]:
def pointpos(x,low_num,high_num):
    if x['pivot']==1:
        return x['Low']-low_num
    elif x['pivot']==2:
        return x['High']+high_num
    else:
        return np.nan

def shortpointpos(x,low_num,high_num):
    if x['shortpivot']==1:
        return x['Low']-low_num
    elif x['shortpivot']==2:
        return x['High']+high_num
    else:
        return np.nan
    
df['pointpos'] = df.apply(lambda row: pointpos(row,2.0,2.0), axis=1)
df['shortpointpos'] = df.apply(lambda row: shortpointpos(row,2.0,2.0), axis=1)

In [26]:
df = df[df['spread'] != 0]
df

,time,Open,High,Low,Close,tick_volume,spread,real_volume
0,1612141200,1862.93,1866.92,1851.62,1853.43,1232,36,0
1,1612142100,1853.53,1859.75,1851.94,1857.41,810,36,0
2,1612143000,1857.38,1859.85,1855.34,1856.60,852,36,0
3,1612143900,1856.83,1857.75,1854.97,1855.83,810,36,0
4,1612144800,1855.61,1856.31,1852.00,1853.02,881,36,0
...,...,...,...,...,...,...,...,...
21924,1641302100,1804.10,1804.41,1802.27,1803.10,863,35,0
21925,1641303000,1803.12,1803.80,1802.34,1803.28,857,35,0
21926,1641303900,1803.26,1804.24,1802.86,1803.89,817,35,0
21927,1641304800,1803.88,1804.88,1801.53,1801.90,1069,35,0


In [ ]:
import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from datetime import datetime
# dfpl = df
# fig = go.Figure(data=[go.Candlestick(x=dfpl.time,
#                 open=dfpl['Open'],
#                 high=dfpl['High'],
#                 low=dfpl['Low'],
#                 close=dfpl['Close'])])

# fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
#                 marker=dict(size=5, color="MediumPurple"),
#                 name="pivot")
# fig.add_scatter(x=dfpl.index, y=dfpl['shortpointpos'], mode="markers",
#                 marker=dict(size=5, color="red"),
#                 name="shortpivot")
# fig.update_layout(xaxis_rangeslider_visible=False)
# fig.show()
b = df.iloc[230:248].copy()
vsdf=b

fig = go.Figure(data=[go.Candlestick(
    x=vsdf['time'],
    open=vsdf['Open'],
    high=vsdf['High'],
    low=vsdf['Low'],
    close=vsdf['Close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])
# fig.add_scatter(x=vsdf.time, y=vsdf['pointpos'], mode="markers",marker=dict(size=5, color="MediumPurple"),name="pivot")
# fig.add_scatter(x=vsdf.time, y=vsdf['shortpointpos'], mode="markers",marker=dict(size=5, color="red"),name="shortpivot")


# Show the chart
fig.show()

In [ ]:
from scipy.stats import linregress
backcandles= 14 #!!!should be less than pivot candles

lengthcheck=2.5
length_=3.0

for candleid in range(len(df)-backcandles):
    if df.iloc[candleid].pivot != 2 or df.iloc[candleid].shortpivot != 2:
        continue
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    minbcount=0 #minimas before head
    maxbcount=0 #maximas before head
    minacount=0 #minimas after head
    maxacount=0 #maximas after head
    
    for i in range(candleid-backcandles, candleid+backcandles):
        if df.iloc[i].shortpivot == 1:
            minim = np.append(minim, df.iloc[i].Low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name             
            if i < candleid:
                minbcount=+1
            elif i>candleid:
                minacount+=1
        if df.iloc[i].shortpivot == 2:
            maxim = np.append(maxim, df.iloc[i].High)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
            if i < candleid:
                maxbcount+=1
            elif i>candleid:
                maxacount+=1
    
    if minbcount<1 or minacount<1 or maxbcount<1 or maxacount<1:
        continue

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    headidx = np.argmax(maxim, axis=0)
    if maxim[headidx]-maxim[headidx-1]>lengthcheck and maxim[headidx]-maxim[headidx+1]>lengthcheck and abs(slmin)<=length_ and xxmin[0]>xxmax[headidx-1] and xxmin[1]<xxmax[headidx+1]:# and (maxim[headidx]-maxim[headidx+1])>(maxim[headidx+1]-minim[headidx+1]) and (maxim[headidx]-maxim[headidx-1])>(maxim[headidx-1]-minim[headidx-1]) :
        print('here is a pattern',minbcount,minacount,maxbcount,maxacount, slmin, candleid)
        #print(maxim)
        #print(xxmax)
        #print(minim)
        #print(xxmin)
        
        
    if candleid % 1000 == 0:
        print(candleid)

In [ ]:
# b = df.iloc[30065:30200+1].copy()



In [13]:
def size(close,open_):
    sz=close-open_
    if sz<0:
        csz=abs(sz)
        candle={'red':csz}
    else:
        candle={'green':sz}
        
    return candle



In [27]:
df['size'] = df.apply(lambda x: size(x['Close'], x['Open']), axis=1)
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_5480\1171017472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = df.apply(lambda x: size(x['Close'], x['Open']), axis=1)


,time,Open,High,Low,Close,tick_volume,spread,real_volume,size
0,1612141200,1862.93,1866.92,1851.62,1853.43,1232,36,0,{'red': 9.5}
1,1612142100,1853.53,1859.75,1851.94,1857.41,810,36,0,{'green': 3.880000000000109}
2,1612143000,1857.38,1859.85,1855.34,1856.60,852,36,0,{'red': 0.7800000000002001}
3,1612143900,1856.83,1857.75,1854.97,1855.83,810,36,0,{'red': 1.0}
4,1612144800,1855.61,1856.31,1852.00,1853.02,881,36,0,{'red': 2.589999999999918}
...,...,...,...,...,...,...,...,...,...
21924,1641302100,1804.10,1804.41,1802.27,1803.10,863,35,0,{'red': 1.0}
21925,1641303000,1803.12,1803.80,1802.34,1803.28,857,35,0,{'green': 0.16000000000008185}
21926,1641303900,1803.26,1804.24,1802.86,1803.89,817,35,0,{'green': 0.6300000000001091}
21927,1641304800,1803.88,1804.88,1801.53,1801.90,1069,35,0,{'red': 1.9800000000000182}


In [28]:
df['reg'] = df['size'].apply(lambda x: 1 if list(x.values())[0] > 6.0 else 0)
df


C:\Users\DELL\AppData\Local\Temp\ipykernel_5480\2111574620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reg'] = df['size'].apply(lambda x: 1 if list(x.values())[0] > 6.0 else 0)


,time,Open,High,Low,Close,tick_volume,spread,real_volume,size,reg
0,1612141200,1862.93,1866.92,1851.62,1853.43,1232,36,0,{'red': 9.5},1
1,1612142100,1853.53,1859.75,1851.94,1857.41,810,36,0,{'green': 3.880000000000109},0
2,1612143000,1857.38,1859.85,1855.34,1856.60,852,36,0,{'red': 0.7800000000002001},0
3,1612143900,1856.83,1857.75,1854.97,1855.83,810,36,0,{'red': 1.0},0
4,1612144800,1855.61,1856.31,1852.00,1853.02,881,36,0,{'red': 2.589999999999918},0
...,...,...,...,...,...,...,...,...,...,...
21924,1641302100,1804.10,1804.41,1802.27,1803.10,863,35,0,{'red': 1.0},0
21925,1641303000,1803.12,1803.80,1802.34,1803.28,857,35,0,{'green': 0.16000000000008185},0
21926,1641303900,1803.26,1804.24,1802.86,1803.89,817,35,0,{'green': 0.6300000000001091},0
21927,1641304800,1803.88,1804.88,1801.53,1801.90,1069,35,0,{'red': 1.9800000000000182},0


In [29]:
# df[df['reg'] == 1]['reg'].value_counts()
# index_list = df[df['reg'] == 1].index.tolist()
# print(index_list)
index_list = []
for i in df[df['reg'] == 1].index:
    size_dict = df.loc[i, 'size']
    size_color = list(size_dict.keys())[0]
    index_list.append((i, size_color))
    
print(index_list)


[(0, 'red'), (36, 'green'), (37, 'red'), (149, 'red'), (316, 'red'), (336, 'red'), (338, 'red'), (340, 'green'), (404, 'green'), (426, 'green'), (702, 'green'), (807, 'red'), (895, 'green'), (1051, 'red'), (1054, 'red'), (1060, 'green'), (1065, 'red'), (1067, 'red'), (1088, 'red'), (1145, 'red'), (1280, 'red'), (1331, 'green'), (1426, 'green'), (1518, 'red'), (1607, 'red'), (1704, 'red'), (1758, 'red'), (1795, 'red'), (1796, 'red'), (1797, 'red'), (1798, 'green'), (1878, 'red'), (2064, 'red'), (2074, 'red'), (2075, 'green'), (2179, 'red'), (2180, 'green'), (2252, 'green'), (2254, 'red'), (2621, 'red'), (2708, 'red'), (2901, 'green'), (3008, 'green'), (3057, 'red'), (3058, 'red'), (3080, 'red'), (3541, 'green'), (3726, 'red'), (3911, 'green'), (4642, 'green'), (4676, 'green'), (4832, 'green'), (5199, 'green'), (5385, 'red'), (5748, 'red'), (5751, 'red'), (6032, 'red'), (6033, 'green'), (6210, 'green'), (6298, 'green'), (6303, 'green'), (6483, 'red'), (6487, 'green'), (6577, 'red'), (658

In [30]:
# # Create a new dataframe with the required columns
# new_df = pd.DataFrame(columns=['index', 'prev1', 'prev2', 'prev3', 'prev4', 'prev5', 'prev6', 'prev7', 'prev8', 'prev9', 'prev10'])

# # Loop through each index in index_list
# for i, _ in index_list:
#     # Get the previous 10 volumes
#     prev_volumes = df.loc[i-10:i-1, 'Volume'].values[::-1]
#     # Add the index and previous volumes to the new dataframe
#     new_df = new_df.append({'index': i, 'prev1': prev_volumes[0], 'prev2': prev_volumes[1], 'prev3': prev_volumes[2], 'prev4': prev_volumes[3], 'prev5': prev_volumes[4], 'prev6': prev_volumes[5], 'prev7': prev_volumes[6], 'prev8': prev_volumes[7], 'prev9': prev_volumes[8], 'prev10': prev_volumes[9]}, ignore_index=True)

# # Print the new dataframe
# print(new_df)
# new_df = pd.DataFrame(columns=['index_','colourM','prev1', 'prev2', 'prev3', 'prev4', 'prev5', 'prev6', 'prev7', 'prev8', 'prev9', 'prev10'])
# for item in index_list:
#     index=item[0]
#     colourmain=item[1]
#     adj_range=10
#     for i in range(adj_range):
#         previous=df.iloc[index-i]
#         size=previous.size
    
#         size_color = list(size_dict.keys())[0]
#         size_number = list(size_dict.values())[0]
#         volume=previous.tick_volume
#         new_df=new_df.append({'index_':index,'colourM':colourmain,f'prev{i}':{'colour':size_color,'size':size_number,'tick_volume':volume}})
        
new_data = []

for item in index_list:
    index = item[0]
    colourmain = item[1]
    adj_range = 10
    mvalue=df.iloc[index]
    sized=mvalue['size']
    msize = list(sized.values())[0]
    volumeM=mvalue.tick_volume
    
    
    prev_volumes = []
    for i in range(adj_range):
        if index - i >= 0:  # To ensure the index is within the range of the DataFrame
            previous = df.iloc[index - i]
            size_dict = previous['size']  # You need to replace this with your own logic
            size_color = list(size_dict.keys())[0]
            size_number = list(size_dict.values())[0]
            volume = previous.tick_volume
            prev_volumes.append({'colour': size_color, 'size': size_number, 'tick_volume': volume})
        else:
            prev_volumes.append(None)
    new_data.append({'index_': index, 'colourM': colourmain,'volumeM':volumeM,'sizeM':msize, 'prev1': prev_volumes[0], 'prev2': prev_volumes[1], 'prev3': prev_volumes[2], 'prev4': prev_volumes[3], 'prev5': prev_volumes[4], 'prev6': prev_volumes[5], 'prev7': prev_volumes[6], 'prev8': prev_volumes[7], 'prev9': prev_volumes[8], 'prev10': prev_volumes[9]})

new_df = pd.DataFrame(new_data)
new_df

,index_,colourM,volumeM,sizeM,prev1,prev2,prev3,prev4,prev5,prev6,prev7,prev8,prev9,prev10
0,0,red,1232,9.50,"{'colour': 'red', 'size': 9.5, 'tick_volume': ...",None,None,None,None,None,None,None,None,None
1,36,green,1405,8.47,"{'colour': 'green', 'size': 8.470000000000027,...","{'colour': 'red', 'size': 2.4300000000000637, ...","{'colour': 'green', 'size': 1.2999999999999545...","{'colour': 'green', 'size': 0.0499999999999545...","{'colour': 'red', 'size': 2.990000000000009, '...","{'colour': 'green', 'size': 0.2699999999999818...","{'colour': 'red', 'size': 0.8799999999998818, ...","{'colour': 'green', 'size': 2.7799999999999727...","{'colour': 'green', 'size': 3.619999999999891,...","{'colour': 'green', 'size': 1.8900000000001, '..."
2,37,red,1367,9.06,"{'colour': 'red', 'size': 9.059999999999945, '...","{'colour': 'green', 'size': 8.470000000000027,...","{'colour': 'red', 'size': 2.4300000000000637, ...","{'colour': 'green', 'size': 1.2999999999999545...","{'colour': 'green', 'size': 0.0499999999999545...","{'colour': 'red', 'size': 2.990000000000009, '...","{'colour': 'green', 'size': 0.2699999999999818...","{'colour': 'red', 'size': 0.8799999999998818, ...","{'colour': 'green', 'size': 2.7799999999999727...","{'colour': 'green', 'size': 3.619999999999891,..."
3,149,red,1413,10.35,"{'colour': 'red', 'size': 10.349999999999909, ...","{'colour': 'green', 'size': 0.9800000000000182...","{'colour': 'red', 'size': 0.7100000000000364, ...","{'colour': 'green', 'size': 0.9600000000000364...","{'colour': 'red', 'size': 2.7200000000000273, ...","{'colour': 'green', 'size': 3.9300000000000637...","{'colour': 'red', 'size': 3.1000000000001364, ...","{'colour': 'red', 'size': 1.599999999999909, '...","{'colour': 'red', 'size': 1.990000000000009, '...","{'colour': 'red', 'size': 0.12999999999988177,..."
4,316,red,1224,7.76,"{'colour': 'red', 'size': 7.759999999999991, '...","{'colour': 'red', 'size': 0.08000000000015461,...","{'colour': 'green', 'size': 0.1400000000001000...","{'colour': 'red', 'size': 2.2799999999999727, ...","{'colour': 'red', 'size': 1.009999999999991, '...","{'colour': 'red', 'size': 0.5699999999999363, ...","{'colour': 'green', 'size': 0.3399999999999181...","{'colour': 'green', 'size': 2.009999999999991,...","{'colour': 'red', 'size': 0.23999999999978172,...","{'colour': 'red', 'size': 0.10999999999989996,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,19754,red,2197,3.41,"{'colour': 'red', 'size': 3.410000000000082, '...","{'colour': 'red', 'size': 4.6099999999999, 'ti...","{'colour': 'red', 'size': 7.369999999999891, '...","{'colour': 'red', 'size': 10.029999999999973, ...","{'colour': 'red', 'size': 5.539999999999964, '...","{'colour': 'green', 'size': 5.360000000000127,...","{'colour': 'green', 'size': 5.680000000000064,...","{'colour': 'green', 'size': 1.4700000000000273...","{'colour': 'red', 'size': 1.009999999999991, '...","{'colour': 'green', 'size': 0.7000000000000455..."
157,19806,green,893,1.92,"{'colour': 'red', 'size': 1.9200000000000728, ...","{'colour': 'red', 'size': 2.2200000000000273, ...","{'colour': 'green', 'size': 9.010000000000218,...","{'colour': 'green', 'size': 1.4399999999998272...","{'colour': 'green', 'size': 0.2400000000000091...","{'colour': 'green', 'size': 0.3900000000001000...","{'colour': 'green', 'size': 0.2300000000000182...","{'colour': 'green', 'size': 1.009999999999991,...","{'colour': 'green', 'size': 0.790000000000191,...","{'colour': 'red', 'size': 0.4600000000000364, ..."
158,20392,green,1730,0.52,"{'colour': 'red', 'size': 0.5199999999999818, ...","{'colour': 'red', 'size': 5.1099999999999, 'ti...","{'colour': 'green', 'size': 6.600000000000136,...","{'colour': 'red', 'size': 1.8600000000001273, ...","{'colour': 'red', 'size': 0.4700000000000273, ...","{'colour': 'green', 'size': 0.9800000000000182...","{'colour': 'green', 'size': 1.3900000000001, '...","{'colour': 'red', 'size': 0.5799999999999272, ...","{'colour': 'red', 'size': 1.3499999999999

In [31]:
foward_data = []

for item in index_list:
    index = item[0]
    colourmain = item[1]
    adj_range = 10
    front_volumes = []
    mvalue=df.iloc[index]
    sized=mvalue['size']
    volumeM=mvalue.tick_volume
    msize = list(sized.values())[0]
    for i in range(adj_range):
        if index - i >= 0:  # To ensure the index is within the range of the DataFrame
            foward = df.iloc[index + i]
            size_dict = foward['size']  # You need to replace this with your own logic
            size_color = list(size_dict.keys())[0]
            size_number = list(size_dict.values())[0]
            volume = foward.tick_volume
            front_volumes.append({'colour': size_color, 'size': size_number, 'tick_volume': volume})
        else:
            front_volumes.append(None)
    foward_data.append({'index_': index, 'colourM': colourmain,'sizeM':msize,'volumeM':volumeM, 'f1': front_volumes[0], 'f2': front_volumes[1], 'f3': front_volumes[2], 'f4': front_volumes[3], 'f5': front_volumes[4], 'f6': front_volumes[5], 'f7': front_volumes[6], 'f8': front_volumes[7], 'f9': front_volumes[8], 'f10': front_volumes[9]})
    
front_df = pd.DataFrame(foward_data)
front_df

,index_,colourM,sizeM,volumeM,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,0,red,9.50,1232,"{'colour': 'red', 'size': 9.5, 'tick_volume': ...",None,None,None,None,None,None,None,None,None
1,36,green,8.47,1405,"{'colour': 'green', 'size': 8.470000000000027,...","{'colour': 'red', 'size': 9.059999999999945, '...","{'colour': 'green', 'size': 3.009999999999991,...","{'colour': 'red', 'size': 1.25, 'tick_volume':...","{'colour': 'red', 'size': 0.36000000000012733,...","{'colour': 'green', 'size': 2.7799999999999727...","{'colour': 'red', 'size': 2.9500000000000455, ...","{'colour': 'red', 'size': 0.32999999999992724,...","{'colour': 'red', 'size': 1.1000000000001364, ...","{'colour': 'red', 'size': 0.8500000000001364, ..."
2,37,red,9.06,1367,"{'colour': 'red', 'size': 9.059999999999945, '...","{'colour': 'green', 'size': 3.009999999999991,...","{'colour': 'red', 'size': 1.25, 'tick_volume':...","{'colour': 'red', 'size': 0.36000000000012733,...","{'colour': 'green', 'size': 2.7799999999999727...","{'colour': 'red', 'size': 2.9500000000000455, ...","{'colour': 'red', 'size': 0.32999999999992724,...","{'colour': 'red', 'size': 1.1000000000001364, ...","{'colour': 'red', 'size': 0.8500000000001364, ...","{'colour': 'green', 'size': 0.2100000000000363..."
3,149,red,10.35,1413,"{'colour': 'red', 'size': 10.349999999999909, ...","{'colour': 'red', 'size': 0.4099999999998545, ...","{'colour': 'green', 'size': 5.230000000000018,...","{'colour': 'red', 'size': 2.7000000000000455, ...","{'colour': 'green', 'size': 4.440000000000055,...","{'colour': 'red', 'size': 4.439999999999827, '...","{'colour': 'red', 'size': 1.3600000000001273, ...","{'colour': 'red', 'size': 0.650000000000091, '...","{'colour': 'green', 'size': 2.730000000000018,...","{'colour': 'red', 'size': 2.589999999999918, '..."
4,316,red,7.76,1224,"{'colour': 'red', 'size': 7.759999999999991, '...","{'colour': 'green', 'size': 1.3100000000001728...","{'colour': 'green', 'size': 0.7100000000000364...","{'colour': 'red', 'size': 1.3099999999999454, ...","{'colour': 'red', 'size': 1.3400000000001455, ...","{'colour': 'red', 'size': 1.6199999999998909, ...","{'colour': 'green', 'size': 2.619999999999891,...","{'colour': 'red', 'size': 2.3299999999999272, ...","{'colour': 'red', 'size': 0.34999999999990905,...","{'colour': 'green', 'size': 0.2000000000000454..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,19754,red,3.41,2197,"{'colour': 'red', 'size': 3.410000000000082, '...","{'colour': 'red', 'size': 3.769999999999982, '...","{'colour': 'green', 'size': 1.4500000000000455...","{'colour': 'red', 'size': 2.619999999999891, '...","{'colour': 'red', 'size': 0.029999999999972715...","{'colour': 'green', 'size': 2.0299999999999727...","{'colour': 'green', 'size': 0.7599999999999909...","{'colour': 'red', 'size': 0.3300000000001546, ...","{'colour': 'green', 'size': 1.150000000000091,...","{'colour': 'red', 'size': 1.099999999999909, '..."
157,19806,green,1.92,893,"{'colour': 'red', 'size': 1.9200000000000728, ...","{'colour': 'green', 'size': 1.5299999999999727...","{'colour': 'red', 'size': 0.060000000000172804...","{'colour': 'red', 'size': 0.8300000000001546, ...","{'colour': 'red', 'size': 1.1799999999998363, ...","{'colour': 'green', 'size': 0.8699999999998909...","{'colour': 'red', 'size': 1.9400000000000546, ...","{'colour': 'red', 'size': 4.370000000000118, '...","{'colour': 'red', 'size': 0.8799999999998818, ...","{'colour': 'red', 'size': 0.6300000000001091, ..."
158,20392,green,0.52,1730,"{'colour': 'red', 'size': 0.5199999999999818, ...","{'colour': 'red', 'size': 1.25, 'tick_volume':...","{'colour': 'green', 'size': 1.8099999999999454...","{'colour': 'green', 'size': 2.0700000000001637...","{'colour': 'red', 'size': 0.03000000000020009,...","{'colour': 'red', 'size': 0.029999999999972715...","{'colour': 'red', 'size': 2.4300000000000637, ...","{'colour': 'red', 'size': 1.4700000000000273, ...","{'colour': 'green', 'size': 0.7699999999999818...","{'colour': 'green',

In [32]:
# import numpy as np
# from sklearn.preprocessing import MinMaxScaler

# Create a scaler object
# scaler = MinMaxScaler()

# Normalize the volumeM and sizeM columns
# front_df['volumeM_normalized'] = scaler.fit_transform(front_df[['volumeM']])

# Define a function to normalize a column using the Min-Max scaling method
def min_max_scale(column):
    return (column - np.min(column)) / (np.max(column) - np.min(column))

# Normalize the volumeM and sizeM columns
front_df['volumeM_normalized'] = min_max_scale(front_df['volumeM'])
front_df['sizeM_normalized'] = min_max_scale(front_df['sizeM'])


In [33]:
front_df[['volumeM_normalized', 'sizeM_normalized']].corr()


,volumeM_normalized,sizeM_normalized
volumeM_normalized,1.000000,0.061707
sizeM_normalized,0.061707,1.000000


In [ ]:
# Extract the values from the size dictionary and create a new column
# df['size_value'] = df['size'].apply(lambda x: list(x.values())[0])

# Calculate the correlation between tick_volume and size_value
corr =front_df['volumeM'].corr(front_df['sizeM'])

print(f"The correlation between tick_volume and size_value is: {corr}")


In [ ]:
orr = df['tick_volume'].corr(df['size_value'])

print(f"The correlation between tick_volume and size_value is: {orr}")